In [ ]:
from queue import PriorityQueue

class Device:
    def __init__(self, id, name, price, quantity, year):
        self.id = id
        self.name = name
        self.price = float(price)
        self.quantity = int(quantity)
        self.year = int(year)
    
    def __str__(self):
        return f"{self.id},{self.name},{self.price},{self.quantity},{self.year}"

class Node:
    def __init__(self, device):
        self.device = device
        self.left = None
        self.right = None

class BST:
    def __init__(self):
        self.root = None
    
    def insert(self, device):
        if not self.root:
            self.root = Node(device)
        else:
            self._insert_recursive(self.root, device)
    
    def _insert_recursive(self, node, device):
        if device.id < node.device.id:
            if node.left is None:
                node.left = Node(device)
            else:
                self._insert_recursive(node.left, device)
        else:
            if node.right is None:
                node.right = Node(device)
            else:
                self._insert_recursive(node.right, device)
    
    def read_file(self, filename):
        try:
            with open(filename, 'r', encoding='utf-8') as file:
                for line in file:
                    id, name, price, quantity, year = line.strip().split(',')
                    device = Device(id, name, price, quantity, year)
                    self.insert(device)
            return True
        except Exception as e:
            print(f"Error reading file: {e}")
            return False
    
    def traverse_NLR(self, node=None):
        if node is None:
            node = self.root
        if node:
            print(str(node.device))
            self.traverse_NLR(node.left)
            self.traverse_NLR(node.right)
    
    def traverse_LNR(self, node=None):
        if node is None:
            node = self.root
        if node:
            self.traverse_LNR(node.left)
            print(str(node.device))
            self.traverse_LNR(node.right)
    
    def traverse_LRN(self, node=None):
        if node is None:
            node = self.root
        if node:
            self.traverse_LRN(node.left)
            self.traverse_LRN(node.right)
            print(str(node.device))
    
    def count_one_child(self, node=None):
        if node is None:
            node = self.root
        if not node:
            return 0
        count = 0
        if (node.left and not node.right) or (node.right and not node.left):
            count = 1
        return count + self.count_one_child(node.left) + self.count_one_child(node.right)
    
    def find_devices_above_price(self, price):
        queue = []
        def traverse(node):
            if node:
                if node.device.price > price:
                    queue.append(node.device)
                traverse(node.left)
                traverse(node.right)
        traverse(self.root)
        return queue
    
    def find_k_most_expensive(self, k):
        devices = []
        def traverse(node):
            if node:
                devices.append(node.device)
                traverse(node.left)
                traverse(node.right)
        traverse(self.root)
        return sorted(devices, key=lambda x: x.price, reverse=True)[:k]
    
    def delete_node(self, id):
        self.root = self._delete_recursive(self.root, id)
    
    def _delete_recursive(self, node, id):
        if not node:
            return None
        
        if id < node.device.id:
            node.left = self._delete_recursive(node.left, id)
        elif id > node.device.id:
            node.right = self._delete_recursive(node.right, id)
        else:
            if not node.left:
                return node.right
            elif not node.right:
                return node.left
            
            min_node = self._find_min(node.right)
            node.device = min_node.device
            node.right = self._delete_recursive(node.right, min_node.device.id)
        return node
    
    def _find_min(self, node):
        current = node
        while current.left:
            current = current.left
        return current
    
    def find_two_child_nodes(self):
        result = []
        def traverse(node):
            if node:
                if node.left and node.right:
                    result.append(node.device)
                traverse(node.left)
                traverse(node.right)
        traverse(self.root)
        return result
    
    def update_device(self, id, new_price, new_quantity):
        def update(node):
            if not node:
                return False
            if node.device.id == id:
                node.device.price = float(new_price)
                node.device.quantity = int(new_quantity)
                return True
            return update(node.left) or update(node.right)
        return update(self.root)
    
    def save_to_file(self, filename):
        devices = []
        def traverse_LNR(node):
            if node:
                traverse_LNR(node.left)
                devices.append(str(node.device))
                traverse_LNR(node.right)
        
        traverse_LNR(self.root)
        try:
            with open(filename, 'w', encoding='utf-8') as file:
                for device in devices:
                    file.write(f"{device}\n")
            return True
        except Exception as e:
            print(f"Error writing to file: {e}")
            return False

def main():
    bst = BST()
    while True:
        print("\n=== QUẢN LÝ THIẾT BỊ ĐIỆN TỬ ===")
        print("1. Đọc file thiết bị")
        print("2. Duyệt cây (NLR)")
        print("3. Duyệt cây (LNR)")
        print("4. Duyệt cây (LRN)")
        print("5. Đếm số nút có 1 con")
        print("6. Tìm thiết bị có giá > x")
        print("7. Tìm k thiết bị có giá cao nhất")
        print("8. Xóa thiết bị theo mã")
        print("9. Tìm nút có 2 con và lưu file")
        print("10. Cập nhật thông tin thiết bị")
        print("11. Ghi dữ liệu ra file")
        print("12. Thoát")
        
        choice = input("Nhập lựa chọn: ")
        
        if choice == '1':
            if bst.read_file("devices.txt"):
                print("Đọc file thành công!")
        
        elif choice == '2':
            print("\nDuyệt cây theo thứ tự NLR:")
            bst.traverse_NLR()
        
        elif choice == '3':
            print("\nDuyệt cây theo thứ tự LNR:")
            bst.traverse_LNR()
        
        elif choice == '4':
            print("\nDuyệt cây theo thứ tự LRN:")
            bst.traverse_LRN()
        
        elif choice == '5':
            count = bst.count_one_child()
            print(f"\nSố nút có 1 con: {count}")
        
        elif choice == '6':
            x = float(input("Nhập giá x: "))
            devices = bst.find_devices_above_price(x)
            print("\nThiết bị có giá > x:")
            for device in devices:
                print(str(device))
        
        elif choice == '7':
            k = int(input("Nhập số lượng k: "))
            devices = bst.find_k_most_expensive(k)
            print(f"\n{k} thiết bị có giá cao nhất:")
            for device in devices:
                print(str(device))
        
        elif choice == '8':
            id = input("Nhập mã thiết bị cần xóa: ")
            bst.delete_node(id)
            print("Đã xóa thiết bị!")
        
        elif choice == '9':
            nodes = bst.find_two_child_nodes()
            with open("NodesWithTwoChild.txt", 'w', encoding='utf-8') as file:
                for device in nodes:
                    file.write(f"{str(device)}\n")
            print("Đã lưu các nút có 2 con vào file!")
        
        elif choice == '10':
            id = input("Nhập mã thiết bị: ")
            price = input("Nhập giá mới: ")
            quantity = input("Nhập số lượng mới: ")
            if bst.update_device(id, price, quantity):
                print("Cập nhật thành công!")
            else:
                print("Không tìm thấy thiết bị!")
        
        elif choice == '11':
            if bst.save_to_file("output.txt"):
                print("Đã ghi dữ liệu ra file!")
        
        elif choice == '12':
            print("Tạm biệt!")
            break
        
        else:
            print("Lựa chọn không hợp lệ!")

if __name__ == "__main__":
    main()